# 1.2 torchtextでデータセットの生成

※　Ubuntuでの動作を前提。

# 1 . 前処理と単語分割の関数を実装


In [27]:
# 単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer

j_t = Tokenizer()


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


In [28]:
# 前処理として正規化をする関数を定義
import re


def preprocessing_text(text):
    # 半角・全角の統一
    # 今回は無視

    # 英語の小文字化
    # 今回はここでは無視
    # output = output.lower()

    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    # 記号と数字の除去
    # 今回は無視。半角記号,数字,英字
    # 今回は無視。全角記号

    # 特定文字を正規表現で置換する
    # 今回は無視

    return text


In [29]:
# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_janome(text)  # Janomeの単語分割

    return ret


# 動作確認
text = "昨日は とても暑く、気温が36度もあった。"
print(tokenizer_with_preprocessing(text))


['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


In [31]:
#文章読み込みテスト
import torchtext

max_length = 255
TEXT = torchtext.data.Field()
LABEL = torchtext.data.Field()


train_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='train.tsv',
    validation='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


In [32]:
print('訓練データの数', len(train_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print(vars(train_ds[0])['Text'])
print(vars(train_ds[0])['Label'])


訓練データの数 1970
1つ目の訓練データ {'Text': ['当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節約志向により、個人消費は本格的な改善には至らず、また少子高齢化、人口減少による社会構造の変化、雇用改善に伴う労働コストの上昇、企業間競争の激化など、引き続き厳しい状況となりました'], 'Label': ['0']}
['当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節約志向により、個人消費は本格的な改善には至らず、また少子高齢化、人口減少による社会構造の変化、雇用改善に伴う労働コストの上昇、企業間競争の激化など、引き続き厳しい状況となりました']
['0']


# 2. 文章データの読み込み

In [33]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリー（単語集：後で解説）に追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を先頭に用意するかどうか
# fix_length：全部の文章を指定した長さと同じになるように、paddingします


In [34]:
# data.TabularDataset 詳細
# https://torchtext.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits

# フォルダ「data」から各tsvファイルを読み込み、Datasetにします
# 1行がTEXTとLABELで区切られていることをfieldsで指示します
train_ds, val_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='train.tsv',
    validation='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])




In [35]:
# 動作確認
# vars(引数）は引数の辞書を返す。
print('訓練データの数', len(train_ds))
print('テストデータの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))
print(vars(val_ds[2])['Text'])

訓練データの数 1970
テストデータの数 843
1つ目の訓練データ {'Text': ['当社', 'グループ', 'を', '取り巻く', '環境', 'は', '、', '実質', '賃金', 'が', '伸び悩む', 'なか', '、', '消費', '者', 'の', '皆様', 'の', '生活', '防衛', '意識', 'の', '高まり', 'や', '節約', '志向', 'により', '、', '個人', '消費', 'は', '本格', '的', 'な', '改善', 'に', 'は', '至ら', 'ず', '、', 'また', '少子', '高齢', '化', '、', '人口', '減少', 'による', '社会', '構造', 'の', '変化', '、', '雇用', '改善', 'に', '伴う', '労働', 'コスト', 'の', '上昇', '、', '企業', '間', '競争', 'の', '激化', 'など', '、', '引き続き', '厳しい', '状況', 'と', 'なり', 'まし', 'た'], 'Label': '0'}
2つ目の訓練データ {'Text': ['春', 'から', '夏', 'にかけまして', 'は', '個人', '消費', 'の', '低迷', 'など', 'により', '、', 'きのこ', 'の', '価格', 'は', '厳しい', '状況', 'で', '推移', 'いたし', 'まし', 'た'], 'Label': '0'}
['また', '、', '世界', '経済', 'において', 'は', '、', '新興', '国', 'の', '経済', '成長', 'が', '鈍化', 'する', 'なか', '、', '米国', 'は', '政権', '交代', 'により', '内向', '化', 'を', '鮮明', 'に', 'し', 'て', 'おり', '、', '英国', 'による', 'ｅｕ', '離脱', '決議', 'など', '不安', '要素', 'が', '散見', 'さ', 'れる', '状況', 'に', 'あり', 'ます']


# 単語の数値化

In [36]:
# ボキャブラリーを作成します
# 訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー（単語集）を構築
TEXT.build_vocab(train_ds, min_freq=1)

# 訓練データ内の単語と頻度を出力(頻度min_freqより大きいものが出力されます)
TEXT.vocab.freqs  # 出力させる


Counter({'当社': 148,
         'グループ': 98,
         'を': 1049,
         '取り巻く': 18,
         '環境': 182,
         'は': 3323,
         '、': 5335,
         '実質': 2,
         '賃金': 1,
         'が': 1683,
         '伸び悩む': 6,
         'なか': 29,
         '消費': 127,
         '者': 45,
         'の': 4246,
         '皆様': 1,
         '生活': 13,
         '防衛': 5,
         '意識': 5,
         '高まり': 25,
         'や': 502,
         '節約': 12,
         '志向': 19,
         'により': 605,
         '個人': 93,
         '本格': 10,
         '的': 124,
         'な': 453,
         '改善': 236,
         'に': 1961,
         '至ら': 6,
         'ず': 33,
         'また': 198,
         '少子': 2,
         '高齢': 7,
         '化': 113,
         '人口': 5,
         '減少': 427,
         'による': 215,
         '社会': 12,
         '構造': 18,
         '変化': 15,
         '雇用': 102,
         '伴う': 95,
         '労働': 3,
         'コスト': 39,
         '上昇': 55,
         '企業': 113,
         '間': 13,
         '競争': 43,
         '激化': 26,
         'など': 411,

In [37]:
# ボキャブラリーの単語をidに変換した結果を出力。
# 頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi  # 出力。string to identifiers 文字列をidへ


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f468f0aa6d8>>,
            {'<unk>': 0,
             '<pad>': 1,
             '、': 2,
             'の': 3,
             '0': 4,
             'は': 5,
             'た': 6,
             '00': 7,
             '円': 8,
             'まし': 9,
             'に': 10,
             'と': 11,
             '万': 12,
             'が': 13,
             '百': 14,
             'し': 15,
             '.': 16,
             '000': 17,
             '億': 18,
             '％': 19,
             'なり': 20,
             '（': 21,
             '）': 22,
             'を': 23,
             '比': 24,
             '売上': 25,
             '利益': 26,
             '高': 27,
             'で': 28,
             '連結': 29,
             '年度': 30,
             '前期': 31,
             '会計': 32,
             'て': 33,
             'により': 34,
             '増': 35,
             ',': 36,
             '減': 37,
             'や': 38,
             '増加': 39,
   

# DataLoaderの作成

In [38]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[ 165,  121,   28,    2,  107,   58,    3,  115,  213,   38,  117,  149,
          238,    3,  155,  411,    2,  189,    3,  253,  199,   46,    3,  369,
           34,    2,  183,    5,  274,  163,   43,   91,   10,   90,   86,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [1854,  159,   52,    0,   44,    5,    2,  241,  315,   71,  382,  285,
           23,  206,   15,    2,  925,  790,   71, 1032,   38,  812, 1211,   71,
          263,    3, 1386,   10,  843,    2,   25,   27,    5,    7,   18,    7,
           14,   12,    8,   21,   41,   48,   24,    7,   16,    4,   19,   37,
           22,   11]]), tensor([35, 50]))
tensor([0, 0])


以上